In [2]:
import requests
import time
# import random
import pandas as pd

In [3]:
url = "https://api.hh.ru/vacancies?area=160&text=python&search_field=name&search_field=company_name&search_field=description&per_page=100"

In [4]:
def get_data_from_url(url):
    data = requests.get(url).json()
    return data


In [5]:
try:
    data = get_data_from_url(url)
    found = data['found']
 
except Exception as e:
    print(e)

print(found,"-------------")
print(data)


198 -------------
{'items': [{'id': '79704747', 'premium': False, 'name': 'IT Интерн (Java, python, C#)', 'department': None, 'has_test': False, 'response_letter_required': False, 'area': {'id': '160', 'name': 'Алматы', 'url': 'https://api.hh.ru/areas/160'}, 'salary': None, 'type': {'id': 'open', 'name': 'Открытая'}, 'address': None, 'response_url': None, 'sort_point_distance': None, 'published_at': '2023-04-24T08:47:21+0300', 'created_at': '2023-04-24T08:47:21+0300', 'archived': False, 'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=79704747', 'insider_interview': None, 'url': 'https://api.hh.ru/vacancies/79704747?host=hh.ru', 'adv_response_url': None, 'alternate_url': 'https://hh.ru/vacancy/79704747', 'relations': [], 'employer': {'id': '23817', 'name': 'Народный банк Казахстана', 'url': 'https://api.hh.ru/employers/23817', 'alternate_url': 'https://hh.ru/employer/23817', 'logo_urls': {'90': 'https://hhcdn.ru/employer-logo/314519.png', '240': 'https://hhcdn

In [6]:
len(data['items'])

100

In [7]:
if found > 100:
    i = 1
    while i*100 <= found:
        temp_url = url + f"&page={i}"
        temp_data = get_data_from_url(temp_url)

        for j in range(len(temp_data['items'])):
            data['items'].append(temp_data['items'][j])
        i+=1
        time.sleep(10)

In [8]:
len(data['items'])

198

In [47]:
from pprint import pprint

# pprint(type(data['items'][0]['salary']))
# print(type(data['items'][0]['snippet']['requirement']))
for item in data['items']:
    if "Интерн" in item['name']:
            pprint(item)

{'accept_incomplete_resumes': True,
 'accept_temporary': False,
 'address': None,
 'adv_response_url': None,
 'alternate_url': 'https://hh.ru/vacancy/79704747',
 'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=79704747',
 'archived': False,
 'area': {'id': '160', 'name': 'Алматы', 'url': 'https://api.hh.ru/areas/160'},
 'contacts': None,
 'created_at': '2023-04-24T08:47:21+0300',
 'department': None,
 'employer': {'alternate_url': 'https://hh.ru/employer/23817',
              'id': '23817',
              'logo_urls': {'240': 'https://hhcdn.ru/employer-logo/387376.png',
                            '90': 'https://hhcdn.ru/employer-logo/314519.png',
                            'original': 'https://hhcdn.ru/employer-logo-original/227106.gif'},
              'name': 'Народный банк Казахстана',
              'trusted': True,
              'url': 'https://api.hh.ru/employers/23817',
              'vacancies_url': 'https://api.hh.ru/vacancies?employer_id=23817'},
 'e

In [42]:
new_data = []
for item in data['items']:
    temp = {
        'salary': None,
        'address': None
    }

    temp['id'] = item['id']
    temp['job_name'] = item['name']
    temp['job_url'] = item['alternate_url']
    temp['area_id'] = item['area']['id']
    temp['area_name'] = item['area']['name']
    temp['employer_id'] = item['employer']['id']
    temp['employer_name'] = item['employer']['name']
    temp['employer_url'] = item['employer']['alternate_url']
    temp['employment_id'] = item['employment']['id']
    temp['employment_name'] = item['employment']['name']
    temp['snippet_requirement'] = item['snippet']['requirement']
    temp['snippet_responsibility'] = item['snippet']['responsibility']
    if item['address'] is not None:
        temp['address'] = {
            'building': item['address']['building'],
            'city': item['address']['city'],
            'id': item['address']['id'],
            'lat': item['address']['lat'],
            'lng': item['address']['lng'],
            'raw': item['address']['raw'],
            'street': item['address']['street'],
            
        }
    # else:
    #     item['address'] = None

    if item['salary'] is not None:
        temp['salary'] = {
            'currency': item['salary']['currency'],
            'from': item['salary']['from'],
            'to': item['salary']['to'],
            'gross': item['salary']['gross'],

        }
    # else:
    #     item['salary'] = None

    new_data.append(temp)
    del temp

len(new_data)

198

In [59]:
new_data[1]

{'salary': None,
 'address': None,
 'id': '79995751',
 'job_name': 'Аналитик',
 'job_url': 'https://hh.ru/vacancy/79995751',
 'area_id': '160',
 'area_name': 'Алматы',
 'employer_id': '3882811',
 'employer_name': 'Next',
 'employer_url': 'https://hh.ru/employer/3882811',
 'employment_id': 'full',
 'employment_name': 'Полная занятость',
 'snippet_requirement': 'Знание формул, сводных таблиц, графиков, диаграмм. Плюсом будет владение (bitrix24, <highlighttext>python</highlighttext>, pandas и другими инструментами data аналитики) (но это...',
 'snippet_responsibility': 'Предоставление ежедневной отчетности. Методы сбора, оценки и анализа информации. Обработка большого массива данных с использованием средств электронной таблицы.'}

In [90]:
import sqlite3

conn = sqlite3.connect('hh_jobs.sqlite')

In [77]:
cursor = conn.cursor()
command = """ CREATE TABLE IF NOT EXISTS
jobs(id INTEGER PRIMARY KEY, job_name TEXT NOT NULL, job_url TEXT NOT NULL, area_id INTEGER,  
employer_id INTEGER, employment_id TEXT, snippet_requirement TEXT, snippet_responsibility TEXT, 
address BOOLEAN NOT NULL, salary BOOLEAN NOT NULL)  """

cursor.execute(command)



In [78]:
command = """ CREATE TABLE IF NOT EXISTS
addresses(job_id INTEGER, building TEXT, city TEXT NOT NULL, address_id INTEGER, 
lat FLOAT, lng FLOAT, raw TEXT NOT NULL, street TEXT, 
FOREIGN KEY(job_id) REFERENCES jobs(id), UNIQUE(job_id))"""
cursor.execute(command)

In [79]:
command = """ CREATE TABLE IF NOT EXISTS
salaries(job_id INTEGER, currency TEXT, from_salary INTEGER, to_salary INTEGER, gross BOOLEAN, 
FOREIGN KEY(job_id) REFERENCES jobs(id), UNIQUE(job_id))  """
cursor.execute(command)

command = """ CREATE TABLE IF NOT EXISTS
areas(area_id INTEGER, area_name TEXT, FOREIGN KEY(area_id) REFERENCES jobs(area_id), UNIQUE(area_id))  """
cursor.execute(command)

command = """ CREATE TABLE IF NOT EXISTS
employers(employer_id INTEGER, employer_name TEXT, employer_url TEXT, 
FOREIGN KEY(employer_id) REFERENCES jobs(employer_id), UNIQUE(employer_id))  """
cursor.execute(command)

command = """ CREATE TABLE IF NOT EXISTS
employments(employment_id, employment_name TEXT, 
FOREIGN KEY(employment_id) REFERENCES jobs(employment_id), UNIQUE(employment_id))"""
cursor.execute(command)

In [87]:
# for item in new_data:
item = new_data[1]
salary_bool = 0 if item['salary'] is None else 1
address_bool = 0 if item['address'] is None else 1
# try:
# command = f""" INSERT INTO jobs VALUES 
# ({item['id']}, {item['job_name']}, {item['job_url']}, {item['area_id']}, {item['employer_id']}, {item['employment_id']},
# {item['snippet_requirement']}, {item['snippet_responsibility']}, {address_bool}, {salary_bool} ) """
command = "INSERT INTO jobs VALUES ("
command += str(item['id']) + "," + item['job_name'] + "," + '/?_!:asdads' +","
command += str(item['area_id'])
# , {item['employer_id']}, {item['employment_id']},
# 'alo,alo', {item['snippet_responsibility']}, {address_bool}, {salary_bool} 
command += ")"
cursor.execute(command)

command = f""" INSERT INTO area VALUES ({item['id']}, {item['area_name']}) """
cursor.execute(command)

command = f""" INSERT INTO employers VALUES ({item['employer_id']}, {item['employer_name']}, {item['employer_url']}) """
# command = f""" INSERT INTO employers VALUES ({item['employer_id']}, NULL, {item['employer_url']}) """
cursor.execute(command)

command = f""" INSERT INTO employments VALUES ({item['employment_id']}, {item['employment_name']}) """
cursor.execute(command)

if address_bool == 1:
    command = f""" INSERT INTO addresses VALUES ({item['id']}, {item['address']['building']}, {item['address']['city']},
    {item['address']['address_id']}, {item['address']['lat']}, {item['address']['lng']}, 
    {item['address']['raw']}, {item['address']['street']}) """
    cursor.execute(command)

if salary_bool == 1:
    gross_bool = 0 if item['salary']['gross'] == False else True
    command = f""" INSERT INTO employments VALUES ({item['id']}, {item['salary']['currency']},
    {item['salary']['from']}, {item['salary']['to']}, {gross_bool}) """
    cursor.execute(command)
# except Exception as e:
#     print(e)
#     pprint(item)
    # break

OperationalError: near "/": syntax error

In [92]:
cursor.close()
conn.close()

ProgrammingError: Cannot operate on a closed database.

In [88]:
import pandas as pd
temp = pd.read_csv("hh_jobs.csv", encoding="utf-8")
temp.head()

,id,job_name,job_url,area_id,area_name,employer_id,employer_name,employer_url,employment_id,employment_name,...,address_city,address_id,address_lat,address_lng,address_raw,address_street,salary_currency,salary_from,salary_to,salary_gross
0,79704747,"IT Интерн (Java, python, C#)",https://hh.ru/vacancy/79704747,160,Алматы,23817,Народный банк Казахстана,https://hh.ru/employer/23817,probation,Стажировка,...,-,-,-,-,-,-,-,-,-,-
1,80065259,Back-end разработчик,https://hh.ru/vacancy/80065259,160,Алматы,5214917,PAYDA Accounting Development (ПАЙДА Аккаунтинг...,https://hh.ru/employer/5214917,full,Полная занятость,...,-,-,-,-,-,-,KZT,500000,NaN,False
2,77435759,Data-аналитик,https://hh.ru/vacancy/77435759,160,Алматы,40662,Колеса,https://hh.ru/employer/40662,full,Полная занятость,...,Алматы,341680,43.243442,76.900208,"Алматы, улица Шевченко, 157Б",улица Шевченко,KZT,800000,1150000,False
3,79995751,Аналитик,https://hh.ru/vacancy/79995751,160,Алматы,3882811,Next,https://hh.ru/employer/3882811,full,Полная занятость,...,Алматы,5728166,43.21877,76.967016,"Алматы, улица Митина, 17",улица Митина,-,-,-,-
4,79795304,Junior разработчик,https://hh.ru/vacancy/79795304,160,Алматы,9629003,SYM TECHNOLOGY,https://hh.ru/employer/9629003,full,Полная занятость,...,-,-,-,-,-,-,-,-,-,-


In [91]:
temp.to_sql("jobs", con=conn)

185

In [ ]:
import psycopg2
from config import *

try:
    conn = psycopg2.connect(
        host=HOST,
        user=USER,
        password=PASSWORD,
        database=DB_NAME        
    )
except:
    pass